# Imports

In [1]:
import win32com.client
import numpy as np
import time
from IPython.display import clear_output

# OLE Python-Simrna
This section was designed to perform tests and understand how to connect Python with Simnra. Each code cell does one thing, and if all cells are executed, it should perform right. For obvious reasons, there is a need to design a script which generates multiple spectra automatically, since this ipynb file needs to be run cell by cell. To execute it automatically, check the last Section

## Simnra.App

### App initialization

In [ ]:
application = win32com.client.Dispatch('Simnra.App')
#application.Reset()
application.Show()

In [ ]:
print('Simrna version: '+application.Version)

## Simnra.Setup

### Setup initialization

In [ ]:
setup = win32com.client.Dispatch('Simnra.Setup')

### Setup settings

In [ ]:
setup.Alpha = np.random.uniform(0,5) #incident angle
time.sleep(0.5)
setup.Energy = np.random.uniform(1400,2000) #beam energy
time.sleep(0.5)
setup.Theta = np.random.uniform(135,165) #exit angle

In [ ]:
print('Incident angle \u03B1 (deg): '+str(setup.Alpha))
print('Energy (keV): '+str(setup.Energy))
print('Scattering angle \u03B8 (deg): '+str(setup.Theta))

## Simnra.Target

### Target initialization


In [ ]:
target = win32com.client.Dispatch('Simnra.Target')

### Target settings

In [ ]:
listofelements= ["Ge","Si","Au"] #to be completed
element = np.random.choice(listofelements)
thick = np.random.uniform(100,500)
print('Element: ' +str(element))
print('Thickness: '+str(thick))

In [ ]:
#ensure that the target only has 1 layer
target.DeleteAllLayer()
time.sleep(0.25)
for i in range(target.TotalNumberOfElements):
    target.DeleteElement(1,i)
    time.sleep(0.25)
target.AddLayer()
time.sleep(0.5)
target.SetLayerThickness(1,thick) #set layer thickness
time.sleep(0.5)
target.AddElement(1) #add element (element has to be defined)
time.sleep(0.5)
target.SetElementName(1,1,element) #set element name
time.sleep(0.5)
target.SetElementConcentration(1,1,1) #set element concentration

In [ ]:
print('Number of layers: '+ str(target.NumberOfLayers))
print('Number of elements: '+ str(target.TotalNumberOfElements))
print('Elements present: '+str([target.ElementName(1,i+1) for i in range(target.TotalNumberOfElements)]))
print('Concentration of elements: '+str([(target.ElementName(1,i+1),target.GetElementConcentration(1,i+1)) for i in range(target.TotalNumberOfElements)]))
print('Target thickness: '+str(target.Thickness))

### Target reset (if needed)

In [ ]:
target.DeleteAllLayer()
for i in range(target.TotalNumberOfElements):
    target.DeleteElement(1,i)
target.AddLayer()

## Simnra.CrossSec

### Cross Section initialization

In [ ]:
crosssection = win32com.client.Dispatch('Simnra.CrossSec')

### Cross Section settings

In [ ]:
crosssection.SelectRutherfordAll()
crosssection.SetEmin(1,0.001)
crosssection.SetEMax(1,4999.999)

In [ ]:
print('CrossSection choosen: '+ str(crosssection.ReactionsChoosen))
print('Number of available cross-section data sets: '+ str(crosssection.Count))

## Simnra.Spectrum

### Spectrum initialization

In [ ]:
spectrum = win32com.client.Dispatch('Simnra.Spectrum')

In [ ]:
spectrum.AutoScale = False
#spectrum.BottomAxisMin = 0
spectrum.BottomAxisMax = 800

In [ ]:
print('Autoscale: '+ str(spectrum.AutoScale))
print('Min: '+str(spectrum.BottomAxisMin))
print('Max: '+str(spectrum.BottomAxisMax))

### Spectrum settings

## Calculate Spectra

In [ ]:
application.CalculateSpectrum()

## Data exporting

In [ ]:
spectra_data=[]
labels_data=[]

In [ ]:
data=spectrum.GetDataArray(2)
data=list(data)
spectra_data.append(data)
aux=[]
aux=[setup.Alpha,setup.Energy,setup.Theta,target.Thickness,[list((target.ElementName(1,i+1),target.GetElementConcentration(1,i+1))) for i in range(target.TotalNumberOfElements)]]
labels_data.append(aux)
print(len(data))

In [ ]:
data = 'spectra_data.txt'
labels = 'labels_data.txt'
with open(data, 'w') as file:
    list_as_string = '\n'.join(map(str, spectra_data))
    file.write(list_as_string)
with open(labels, 'w') as file:
    list_as_string = '\n'.join(map(str, labels_data))
    file.write(list_as_string)

## Reset objects

This is just an idea of how it might be possible to have only one instance of the SIMNRA running for all the spectra generation instead of one instance of the SIMNRA for each spectra generation

In [ ]:
setup,target,spectrum,crosssection = None,None,None,None

# Automatically Spectra Generation
**Note:** don't forget to run the cell that contains all the imports necessary in order to run the code, located in the first Section "Imports".

Generate .txt files where the data will be stored and initialize lists for further use.

**Caution:** running this will overwrite the data.txt and labels.txt files, which can lead into losing all the spectra data!

### Generate files

In [27]:
f= open('labels3.txt', 'w')
f= open('data3.txt', 'w')

## Code

In [15]:
def Dsimnra(application,verbose):

    application.Minimize()

    if verbose == 1:
        print('Simrna version: '+application.Version)


def Dsetup(setup,alpha,energy,theta,verbose):

    #setup settings

    setup.CalibrationLinear = 2.45
    
    setup.Alpha = alpha #incident angle

    setup.Energy = energy #beam energy

    setup.Theta = theta #exit angle

    if verbose == 1:
        print('Incident angle \u03B1 (deg): '+str(setup.Alpha))
        print('Energy (keV): '+str(setup.Energy))
        print('Scattering angle \u03B8 (deg): '+str(setup.Theta))
        

def Dtarget(target,element,thickness,verbose):

    #target settings

    #ensure that the target only has 1 layer

    target.DeleteAllLayer()

    for i in range(target.TotalNumberOfElements):
        target.DeleteElement(1,i)
    target.AddLayer()

    #layer settings

    target.SetLayerThickness(1,thickness) #set layer thickness

    target.AddElement(1) #add empty element

    target.SetElementName(1,1,element) #set element name

    target.SetElementConcentration(1,1,1) #set element concentration

    if verbose == 1:
        print('Number of layers: '+ str(target.NumberOfLayers))
        print('Number of elements: '+ str(target.TotalNumberOfElements))
        print('Elements present: '+str([target.ElementName(1,i+1) for i in range(target.TotalNumberOfElements)]))
        print('Concentration of elements: '+str([(target.ElementName(1,i+1),target.GetElementConcentration(1,i+1)) for i in range(target.TotalNumberOfElements)]))
        print('Target thickness: '+str(target.Thickness))

def Dprojectile(projectile,particle,verbose):

    projectile.Name = particle

    if verbose == 1:
        print('Beam particle: '+particle)

def Dcrossec(crosssection,verbose):

    #cross section settings


    crosssection.SelectRutherfordAll()

    crosssection.SetEmin(1,0.001)
    crosssection.SetEMax(1,4999.999)

    if verbose == 1:
        print('CrossSection choosen: '+ str(crosssection.ReactionsChoosen))
        print('Number of available cross-section data sets: '+ str(crosssection.Count))


def Dspectra(spectrum,verbose):

    spectrum = win32com.client.Dispatch('Simnra.Spectrum')


def Dsimulation(application,spectrum,verbose):

    application.CalculateSpectrum()

    if verbose == 1:
        print('Number of Channels: '+str(spectrum.NumberOfChannels(2)))

In [33]:
def calculate_spectra(iterations,verbose,clear):

    for _ in range(iterations):

        application = win32com.client.Dispatch('Simnra.App')
        setup = win32com.client.Dispatch('Simnra.Setup')
        target = win32com.client.Dispatch('Simnra.Target')
        projectile = win32com.client.Dispatch('Simnra.Projectile')
        crosssection = win32com.client.Dispatch('Simnra.CrossSec')
        spectrum = win32com.client.Dispatch('Simnra.Spectrum')

        #element = np.random.choice(["Pb","Au"]) #pick an element
        element = "Au"
        #thickness = np.random.uniform(650,750) #layer thickness
        thickness = 730
        alpha=0
        #energy=np.random.uniform(1800,2000)
        energy=2000
        #theta=np.random.choice([140,165])
        theta=165
        #particle = np.random.choice(['He','H'])
        particle= 'He'
    

        
        #################################################################

        Dsimnra(application,verbose)

        ################################################################# 

        Dsetup(setup,alpha,energy,theta,verbose)

        #################################################################

        Dtarget(target,element,thickness,verbose)

        #################################################################
        
        Dprojectile(projectile,particle,verbose)

        #################################################################

        Dcrossec(crosssection,verbose)

        #################################################################

        Dspectra(spectrum,verbose)

        #################################################################

        Dsimulation(application,spectrum,verbose)

        #data exporting

        datalist=list(spectrum.GetDataArray(2))
        size=len(datalist)
        while size<2000:
            datalist.append(0.0)
            size+=1


        if particle == 'H':
            particle = 1
        else:
            particle = 0


        if element == 'Pb':
            element = 1
        else:
            element = 0
            

        labelslist=[thickness,particle,energy,theta,element]
        
        with open('data5.txt', 'a') as file:
            list_as_string = ' '.join(map(str, datalist))
            file.write(list_as_string)
            file.write('\n')
        with open('labels5.txt', 'a') as file:
            list_as_string = ' '.join(map(str, labelslist))
            file.write(list_as_string)
            file.write('\n')

        print()
        print()
        if clear == 1:  
            clear_output()

In [34]:
calculate_spectra(iterations=1,verbose=1,clear=1)

# Experimental data treatment

Converts the experimental runs into a format that is readable in SIMNRA. To choose the files, adjust the directory and number of runs.

In [ ]:
for i in range(1,10):
    fi = open('./RBS_Runs/2022_07Set_Au&Pb&C/20220907/RBS1run0'+str(i)+'.dat', 'r')
    fo = open('./RBS_Runs/2022_07Set_Au&Pb&C/20220907/RBS1run0'+str(i)+'_sort.dat',"w")

    i=0
    j=-1
    array = []
    data = []
    for line in fi:
        try:
            array.append([int(x) for x in line.split()])
            for val in array[i]:
                if (j>0): fo.write("%i %i \n" % (j,val))
        #		data[j]= val
                j= j+1
            i = i+1
        except:
            fo.close()

    fo.close()